In [3]:
# %load setup_spm.py
from nipype.interfaces import spm

MATLAB_RUNTIME = '/home/rongfei/MATLAB/runtime/R2024b'
SPM_START_SCRIPT = '/home/rongfei/Builds/spm12/spm_standalone/run_spm25.sh'

MCR_COMMAND = f'{SPM_START_SCRIPT} {MATLAB_RUNTIME} script'
print("MCR_COMMAND: ", MCR_COMMAND)

spm.SPMCommand.set_mlab_paths(MCR_COMMAND, use_mcr=True)
print(spm.SPMCommand().version)

MCR_COMMAND:  /home/rongfei/Builds/spm12/spm_standalone/run_spm25.sh /home/rongfei/MATLAB/runtime/R2024b script


stty: 'standard input': Inappropriate ioctl for device


25.25.01.02


stty: 'standard input': Inappropriate ioctl for device


In [4]:
from nipype.interfaces import fsl

# check FSL integration

print("Check if FSL is runnable... ", end='')
print('ok' if fsl.check_fsl() == 0 else 'not ok')

Check if FSL is runnable... ok


In [97]:
from pathlib import Path
import pandas as pd
import numpy as np
import warnings
import os

# Create a class to handle subject information
class SubjectInfo:
    def __init__(
        self,
        subject_id: str,
        raw_data_dir: Path | None = None,
        derivatives_dir: Path | None = None,
        working_dir: Path | None = None,
        results_dir: Path | None = None,
    ):
        if raw_data_dir is None:
            raw_data_dir = Path("data")
        if derivatives_dir is None:
            derivatives_dir = Path("derivatives")
        if working_dir is None:
            working_dir = Path("working")
        if results_dir is None:
            results_dir = Path("results")

        working_dir.mkdir(parents=True, exist_ok=True)

        self.subject_id: str = subject_id
        self.raw_data_dir: Path = raw_data_dir
        self.derivatives_dir: Path = derivatives_dir
        self.working_dir: Path = working_dir
        self.results_dir: Path = results_dir

    def get_confounds_file(self, task_name: str):
        confounds_file = (
            self.derivatives_dir
            / self.subject_id
            / "func"
            / f"{self.subject_id}_task-{task_name}_bold_confounds.tsv"
        )
        if not confounds_file.exists():
            raise FileNotFoundError(f"Confounds file not found: {confounds_file}")
        return confounds_file

    def get_event_file(self, task_name: str):
        event_file = (
            self.raw_data_dir
            / self.subject_id
            / "func"
            / f"{self.subject_id}_task-{task_name}_events.tsv"
        )
        if not event_file.exists():
            raise FileNotFoundError(f"Event file not found: {event_file}")
        return event_file

    def get_subject_id(self):
        return self.subject_id

    def filter_confounds(self, task_name: str, confounds_columns: list[str]):
        confounds_file = self.get_confounds_file(task_name)
        confounds_df = pd.read_csv(confounds_file, sep="\t", na_values="n/a")
        selected_columns = confounds_df[confounds_columns]

        # convert to float and replace n/a with 0
        selected_columns = selected_columns.map(
            lambda x: float(x) if x != "n/a" else 0
        )

        selected_columns = selected_columns.round(3)

        # get the file name without the extension and add filtered to the end before the extension
        confounds_folder = self.working_dir / self.subject_id
        confounds_folder.mkdir(parents=True, exist_ok=True)

        confounds_file_name = (
            confounds_folder
            / f"{self.subject_id}_task-{task_name}_confounds_filtered.tsv"
        )
        selected_columns.to_csv(confounds_file_name, sep=" ", header=False, index=False)
        return confounds_file_name

    def get_standard_brain_mask(self, task_name: str):
        mask_file = (
            self.derivatives_dir
            / self.subject_id
            / "func"
            / f"{self.subject_id}_task-{task_name}_bold_space-MNI152NLin2009cAsym_brainmask.nii.gz"
        )
        if not mask_file.exists():
            raise FileNotFoundError(f"Mask file not found: {mask_file}")
        return mask_file

    def get_transformed_brain_mask(self, task_name: str):
        mask_file = (
            self.derivatives_dir
            / self.subject_id
            / "func"
            / f"{self.subject_id}_task-{task_name}_bold_space-MNI152NLin2009cAsym_brainmask_processed.nii.gz"
        )
        return mask_file

    def get_processed_bold_file(self, task_name: str):
        bold_file = (
            self.derivatives_dir
            / self.subject_id
            / "func"
            / f"{self.subject_id}_task-{task_name}_bold_space-MNI152NLin2009cAsym_preproc.nii.gz"
        )
        if not bold_file.exists():
            raise FileNotFoundError(f"Bold file not found: {bold_file}")
        return bold_file

    def get_smoothed_bold_file(self, task_name: str):
        bold_file = (
            self.derivatives_dir
            / self.subject_id
            / "func"
            / f"{self.subject_id}_task-{task_name}_bold_space-MNI152NLin2009cAsym_preproc_smoothed.nii.gz"
        )
        return bold_file

    def get_events(self, task_name: str):
        events_file = self.get_event_file(task_name)
        if not events_file.exists():
            raise FileNotFoundError(f"Events file not found: {events_file}")

        events_df = pd.read_csv(events_file, sep="\t", na_values="n/a")
        return events_df
    
    def get_working_task_dir(self, task_name: str):
        task_dir = self.working_dir / self.subject_id / task_name
        task_dir.mkdir(parents=True, exist_ok=True)
        return task_dir
    
    def get_output_dir(self):
        task_dir = self.results_dir / self.subject_id 
        task_dir.mkdir(parents=True, exist_ok=True)
        return task_dir

    def create_fsl_explanatory_variable_file(
        self,
        task_name: str,
        events_df: pd.DataFrame,
        ev_name: str,
        reference_duration_column: str | None = None,
        reference_amplitude_column: str | None = None,
        fixed_duration: float = 1.0,
        fixed_amplitude: float = 1.0,
    ):
        events_df = events_df.dropna(subset=["onset"])
        if events_df.empty:
            warnings.warn(f"No events found in the events file for {ev_name}")
            


        onsets = events_df["onset"].to_list()
        if reference_duration_column is not None:
            durations = events_df[reference_duration_column].to_list()
        else:
            durations = [fixed_duration] * len(onsets)

        if reference_amplitude_column is not None:
            # center the amplitude around 0
            amplitudes = events_df[reference_amplitude_column]
            amplitudes = (
                amplitudes.apply(lambda x: x - np.mean(amplitudes)).round(3).to_list()
            )

        else:
            amplitudes = [fixed_amplitude] * len(onsets)

        fsl_explanatory_variable_df = pd.DataFrame(
            {"0": onsets, "1": durations, "2": amplitudes}
        )

        ev_file_folder = self.working_dir / self.subject_id / task_name 
        ev_file_folder.mkdir(parents=True, exist_ok=True)
        ev_file_path = ev_file_folder / f"{ev_name}.txt"

        fsl_explanatory_variable_df.to_csv(
            ev_file_path,
            sep="\t",
            header=False,
            index=False,
        )

        return ev_file_path


    def create_fsl_evs_scap(self):
        events_df = self.get_events("scap")

        correct_events = events_df[events_df["ResponseAccuracy"] == "CORRECT"]
        incorrect_events = events_df[events_df["ResponseAccuracy"] == "INCORRECT"]

        evs = []
        n_regressors = 25
        orthogonalization = {x: {y:0 for y in range(1,n_regressors+1)} for x in range(1,n_regressors+1)}

        load_levels = [1, 3, 5, 7]
        delay_levels = [1.5, 3.0, 4.5]

        for load_level in load_levels:
            for delay_level in delay_levels:
                condition_mask = (correct_events["Load"] == load_level) & (
                    correct_events["Delay"] == delay_level
                )
                condition_events = correct_events[condition_mask]

                if condition_events.empty:
                    warnings.warn(
                        f"No events found for load {load_level} and delay {delay_level}"
                    )
                
                # condition_events["onset"] = condition_events["onset"] + delay_level
                condition_events.loc[:, "onset"] = condition_events["onset"] + delay_level

                fixed_ev = self.create_fsl_explanatory_variable_file(
                    "scap",
                    condition_events,
                    f"LOAD{load_level}_DELAY{delay_level}",
                    fixed_duration=5.0,
                    fixed_amplitude=1.0,
                )

                reaction_time_ev = self.create_fsl_explanatory_variable_file(
                    "scap",
                    condition_events,
                    f"LOAD{load_level}_DELAY{delay_level}_rt",
                    reference_duration_column="ReactionTime",
                    fixed_amplitude=1.0,
                )

                evs.append(fixed_ev)
                fix_index = len(evs) 

                evs.append(reaction_time_ev)
                reaction_time_index = len(evs)
                orthogonalization[fix_index][reaction_time_index] = 1
                orthogonalization[reaction_time_index][0] = 1


        
        if not incorrect_events.empty:
            error_ev = self.create_fsl_explanatory_variable_file(
                "scap",
                incorrect_events,
                "Error",
                fixed_duration=5.0,
                fixed_amplitude=1.0,
            )
            evs.append(error_ev)
        

        valid_ev_files = []
        for ev_file in evs:
            if os.path.exists(ev_file) and os.path.getsize(ev_file) > 0:
                valid_ev_files.append(ev_file.absolute().as_posix())
            else:
                print(f"Warning: Empty or missing EV file: {ev_file}")
        
        # Validation
        expected_regressors = len(load_levels) * len(delay_levels) * 2 + 1  # 25
        if len(valid_ev_files) != expected_regressors:
            print(f"Warning: Expected {expected_regressors} regressors, got {len(valid_ev_files)}")

        
        return {
            'event_files': valid_ev_files,
            'orthogonalization': orthogonalization
        }


In [73]:
subject_info = SubjectInfo('sub-11106')
subject_info.filter_confounds('scap', ['X', 'Y', 'Z', 'RotX', 'RotY', 'RotZ'])
display(subject_info.get_standard_brain_mask('scap'))
display(subject_info.get_processed_bold_file('scap'))
print(subject_info.create_fsl_evs_scap()['orthogonalization'])
print(subject_info.create_fsl_evs_scap()['event_files'])

PosixPath('derivatives/sub-11106/func/sub-11106_task-scap_bold_space-MNI152NLin2009cAsym_brainmask.nii.gz')

PosixPath('derivatives/sub-11106/func/sub-11106_task-scap_bold_space-MNI152NLin2009cAsym_preproc.nii.gz')

{1: {1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0}, 2: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 0: 1}, 3: {1: 0, 2: 0, 3: 0, 4: 1, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0}, 4: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 0: 1}, 5: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 1, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0}, 6: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 

In [ ]:
def create_scap_contrast():
    contrasts = []

    # All conditions
    contrasts += [
        (
            "All",
            "T",
            [
                "LOAD1_DELAY1.5",
                "LOAD1_DELAY3.0",
                "LOAD1_DELAY4.5",
                "LOAD3_DELAY1.5",
                "LOAD3_DELAY3.0",
                "LOAD3_DELAY4.5",
                "LOAD5_DELAY1.5",
                "LOAD5_DELAY3.0",
                "LOAD5_DELAY4.5",
                "LOAD7_DELAY1.5",
                "LOAD7_DELAY3.0",
                "LOAD7_DELAY4.5",
            ],
            [1] * 12,
        )
    ]

    contrasts += [
        (
            "All_rt",
            "T",
            [
                "LOAD1_DELAY1.5_rt",
                "LOAD1_DELAY3.0_rt",
                "LOAD1_DELAY4.5_rt",
                "LOAD3_DELAY1.5_rt",
                "LOAD3_DELAY3.0_rt",
                "LOAD3_DELAY4.5_rt",
                "LOAD5_DELAY1.5_rt",
                "LOAD5_DELAY3.0_rt",
                "LOAD5_DELAY4.5_rt",
                "LOAD7_DELAY1.5_rt",
                "LOAD7_DELAY3.0_rt",
                "LOAD7_DELAY4.5_rt",
            ],
            [1] * 12,
        )
    ]

    # # Load effects
    contrasts += [
        ("Load1", "T", ["LOAD1_DELAY1.5", "LOAD1_DELAY3.0", "LOAD1_DELAY4.5"], [1] * 3)
    ]
    contrasts += [
        ("Load3", "T", ["LOAD3_DELAY1.5", "LOAD3_DELAY3.0", "LOAD3_DELAY4.5"], [1] * 3)
    ]
    contrasts += [
        ("Load5", "T", ["LOAD5_DELAY1.5", "LOAD5_DELAY3.0", "LOAD5_DELAY4.5"], [1] * 3)
    ]
    contrasts += [
        ("Load7", "T", ["LOAD7_DELAY1.5", "LOAD7_DELAY3.0", "LOAD7_DELAY4.5"], [1] * 3)
    ]

    # # Delay effects
    contrasts += [
        (
            "Delay1.5",
            "T",
            ["LOAD1_DELAY1.5", "LOAD3_DELAY1.5", "LOAD5_DELAY1.5", "LOAD7_DELAY1.5"],
            [1] * 4,
        )
    ]
    contrasts += [
        (
            "Delay3",
            "T",
            ["LOAD1_DELAY3.0", "LOAD3_DELAY3.0", "LOAD5_DELAY3.0", "LOAD7_DELAY3.0"],
            [1] * 4,
        )
    ]
    contrasts += [
        (
            "Delay4.5",
            "T",
            ["LOAD1_DELAY4.5", "LOAD3_DELAY4.5", "LOAD5_DELAY4.5", "LOAD7_DELAY4.5"],
            [1] * 4,
        )
    ]

    # Parametric contrasts
    contrasts += [
        (
            "LinearUp_load",
            "T",
            [
                "LOAD1_DELAY1.5",
                "LOAD1_DELAY3.0",
                "LOAD1_DELAY4.5",
                "LOAD3_DELAY1.5",
                "LOAD3_DELAY3.0",
                "LOAD3_DELAY4.5",
                "LOAD5_DELAY1.5",
                "LOAD5_DELAY3.0",
                "LOAD5_DELAY4.5",
                "LOAD7_DELAY1.5",
                "LOAD7_DELAY3.0",
                "LOAD7_DELAY4.5",
            ],
            [-3] * 3 + [-1] * 3 + [1] * 3 + [3] * 3,
        )
    ]

    contrasts += [
        (
            "LinearUp_delay",
            "T",
            [
                "LOAD1_DELAY1.5",
                "LOAD1_DELAY3.0",
                "LOAD1_DELAY4.5",
                "LOAD3_DELAY1.5",
                "LOAD3_DELAY3.0",
                "LOAD3_DELAY4.5",
                "LOAD5_DELAY1.5",
                "LOAD5_DELAY3.0",
                "LOAD5_DELAY4.5",
                "LOAD7_DELAY1.5",
                "LOAD7_DELAY3.0",
                "LOAD7_DELAY4.5",
            ],
            [-1, 0, 1] * 4,
        )
    ]

    # Pairwise load comparisons
    contrasts += [
        (
            "Load3-load1",
            "T",
            [
                "LOAD1_DELAY1.5",
                "LOAD1_DELAY3.0",
                "LOAD1_DELAY4.5",
                "LOAD3_DELAY1.5",
                "LOAD3_DELAY3.0",
                "LOAD3_DELAY4.5",
            ],
            [-1, -1, -1, 1, 1, 1],
        )
    ]
    contrasts += [
        (
            "Load5-load1",
            "T",
            [
                "LOAD1_DELAY1.5",
                "LOAD1_DELAY3.0",
                "LOAD1_DELAY4.5",
                "LOAD5_DELAY1.5",
                "LOAD5_DELAY3.0",
                "LOAD5_DELAY4.5",
            ],
            [-1, -1, -1, 1, 1, 1],
        )
    ]
    contrasts += [
        (
            "Load7-load1",
            "T",
            [
                "LOAD1_DELAY1.5",
                "LOAD1_DELAY3.0",
                "LOAD1_DELAY4.5",
                "LOAD7_DELAY1.5",
                "LOAD7_DELAY3.0",
                "LOAD7_DELAY4.5",
            ],
            [-1, -1, -1, 1, 1, 1],
        )
    ]
    contrasts += [
        (
            "Load5-load3",
            "T",
            [
                "LOAD3_DELAY1.5",
                "LOAD3_DELAY3.0",
                "LOAD3_DELAY4.5",
                "LOAD5_DELAY1.5",
                "LOAD5_DELAY3.0",
                "LOAD5_DELAY4.5",
            ],
            [-1, -1, -1, 1, 1, 1],
        )
    ]
    contrasts += [
        (
            "Load7-load3",
            "T",
            [
                "LOAD3_DELAY1.5",
                "LOAD3_DELAY3.0",
                "LOAD3_DELAY4.5",
                "LOAD7_DELAY1.5",
                "LOAD7_DELAY3.0",
                "LOAD7_DELAY4.5",
            ],
            [-1, -1, -1, 1, 1, 1],
        )
    ]
    contrasts += [
        (
            "Load7-load5",
            "T",
            [
                "LOAD5_DELAY1.5",
                "LOAD5_DELAY3.0",
                "LOAD5_DELAY4.5",
                "LOAD7_DELAY1.5",
                "LOAD7_DELAY3.0",
                "LOAD7_DELAY4.5",
            ],
            [-1, -1, -1, 1, 1, 1],
        )
    ]

    # Pairwise delay comparisons
    contrasts += [
        (
            "Delay4_5-delay1_5",
            "T",
            [
                "LOAD1_DELAY1.5",
                "LOAD3_DELAY1.5",
                "LOAD5_DELAY1.5",
                "LOAD7_DELAY1.5",
                "LOAD1_DELAY4.5",
                "LOAD3_DELAY4.5",
                "LOAD5_DELAY4.5",
                "LOAD7_DELAY4.5",
            ],
            [-1, -1, -1, -1, 1, 1, 1, 1],
        )
    ]
    contrasts += [
        (
            "Delay3-delay1_5",
            "T",
            [
                "LOAD1_DELAY1.5",
                "LOAD3_DELAY1.5",
                "LOAD5_DELAY1.5",
                "LOAD7_DELAY1.5",
                "LOAD1_DELAY3.0",
                "LOAD3_DELAY3.0",
                "LOAD5_DELAY3.0",
                "LOAD7_DELAY3.0",
            ],
            [-1, -1, -1, -1, 1, 1, 1, 1],
        )
    ]
    contrasts += [
        (
            "Delay4_5-delay3",
            "T",
            [
                "LOAD1_DELAY3.0",
                "LOAD3_DELAY3.0",
                "LOAD5_DELAY3.0",
                "LOAD7_DELAY3.0",
                "LOAD1_DELAY4.5",
                "LOAD3_DELAY4.5",
                "LOAD5_DELAY4.5",
                "LOAD7_DELAY4.5",
            ],
            [-1, -1, -1, -1, 1, 1, 1, 1],
        )
    ]

    # Create bidirectional contrasts (positive and negative)
    bidirectional_contrasts = []
    for name, contrast_type, conditions, weights in contrasts:
        # Add original
        bidirectional_contrasts.append((name, contrast_type, conditions, weights))

        if "-" in name:
            # Reverse: "A-B" becomes "B-A"
            parts = name.split("-", 1)
            neg_name = f"{parts[1]}-{parts[0]}"
        else:
            # Add neg_ prefix
            neg_name = f"neg_{name}"

        bidirectional_contrasts.append(
            (neg_name, contrast_type, conditions, [-w for w in weights])
        )

    contrasts = bidirectional_contrasts
    return contrasts

In [ ]:
# Create a workflow function

import nipype.pipeline.engine as pe
import nipype.interfaces.utility as niu
import nipype.interfaces.fsl as fsl
import nipype.interfaces.afni as afni
from nipype.algorithms.modelgen import SpecifyModel
import shutil



def subject_level_modeling(subject_info: SubjectInfo, task_name='scap'):
    workflow = pe.Workflow(name="first_level")
    
    # Define the input nodes to accept inputs of the following:
    # - functional image
    # - brain mask
    # - confounds file


    # Blur Mask
    in_file = subject_info.get_processed_bold_file(task_name).absolute().as_posix()
    mask_file = subject_info.get_standard_brain_mask(task_name).absolute().as_posix()
    out_file = subject_info.get_processed_bold_file(task_name).absolute().as_posix()

    print(in_file)
    print(mask_file)
    print(out_file)
    apply_mask = pe.Node(
        fsl.maths.ApplyMask(
            in_file=in_file,
            mask_file=mask_file,
            out_file=out_file,
        ),
        name="apply_mask",
    )

    mask_file = subject_info.get_standard_brain_mask(task_name).absolute().as_posix()
    out_file = subject_info.get_smoothed_bold_file(task_name).absolute().as_posix()

    apply_smooth = pe.Node(
        afni.BlurInMask(
            mask=mask_file,
            out_file=out_file,
            fwhm=5.0, # Suggested by original paper
        ),
        name="apply_smooth",
    )

    # Get confounds

    confounds_file = subject_info.filter_confounds(task_name, ['X', 'Y', 'Z', 'RotX', 'RotY', 'RotZ'])
    confounds_file = confounds_file.absolute().as_posix()

    # Get regressors (as event files) and orthogonality matrix
    event_files = subject_info.create_fsl_evs_scap()['event_files']
    orthogonalization = subject_info.create_fsl_evs_scap()['orthogonalization']

    model_specification = pe.Node(
        SpecifyModel(
            event_files=event_files,
            realignment_parameters=confounds_file,
            input_units='secs',
            time_repetition=2.0,
            high_pass_filter_cutoff=100,
        ),
        name="model_specification",
    )

    l1design = pe.Node(
        fsl.Level1Design(
            contrasts=create_scap_contrast(),
            orthogonalization=orthogonalization,
            interscan_interval=2.0,
            bases={'dgamma': {'derivs': True}},
            model_serial_correlations=True,
        ),
        name="l1design",
    )

    l1feat = pe.Node(
        fsl.FEATModel(),
        name="l1feat",
    )

    l1estimate = pe.Node(
        fsl.FEAT(),
        name="l1estimate",
    )

    workflow.base_dir = str(subject_info.get_working_task_dir(task_name))
    workflow.connect([
        (apply_mask, apply_smooth, [('out_file', 'in_file')]),
        (apply_smooth, model_specification, [('out_file', 'functional_runs')]),
        (model_specification, l1design, [('session_info', 'session_info')]),
        (l1design, l1feat, [('fsf_files', 'fsf_file'), ('ev_files', 'ev_files')]),
        (l1design, l1estimate, [('fsf_files', 'fsf_file')]), # Note the plural difference
    ])

    workflow.write_graph(graph2use='colored', format='png', simple_form=True)

    workflow.run('MultiProc')

    working_feat_dir = subject_info.get_working_task_dir(task_name) / 'first_level' / 'l1estimate'/ 'run0.feat'

    results_feat_dir = subject_info.get_output_dir() / 'scap.feat'

    # move working_feat_dir to results_feat_dir
    shutil.move(working_feat_dir, results_feat_dir)











In [104]:
import tqdm

# read control_subjects_ids.txt and adhd_subjects_ids.txt into lists
control_subjects_ids = pd.read_csv('control_subjects_ids.txt', header=None).iloc[:, 0].tolist()
adhd_subjects_ids = pd.read_csv('adhd_subjects_ids.txt', header=None).iloc[:, 0].tolist()

assert len(control_subjects_ids) == len(adhd_subjects_ids)

ids = []
for i in range(len(control_subjects_ids)):
    ids.append(control_subjects_ids[i])
    ids.append(adhd_subjects_ids[i])





In [105]:
subjects = [SubjectInfo(id) for id in ids]

for subject_info in tqdm.tqdm(subjects):
    feat_report = subject_info.get_output_dir() / 'scap.feat' / 'report.html'
    if not feat_report.exists():
        subject_level_modeling(subject_info)
    with open("completed_subjects.txt", 'a') as f:
        f.write(f"{subject_info.subject_id}\n")


  0%|          | 0/70 [00:00<?, ?it/s]

/home/rongfei/WorkSpace/consortium/derivatives/sub-10998/func/sub-10998_task-scap_bold_space-MNI152NLin2009cAsym_preproc.nii.gz
/home/rongfei/WorkSpace/consortium/derivatives/sub-10998/func/sub-10998_task-scap_bold_space-MNI152NLin2009cAsym_brainmask.nii.gz
/home/rongfei/WorkSpace/consortium/derivatives/sub-10998/func/sub-10998_task-scap_bold_space-MNI152NLin2009cAsym_preproc.nii.gz
250917-19:01:40,763 nipype.workflow INFO:
	 Generated workflow graph: working/sub-10998/scap/first_level/graph.png (graph2use=colored, simple_form=True).


/tmp/ipykernel_30419/874155052.py:211: UserWarning: No events found for load 3 and delay 4.5
  warnings.warn(
/tmp/ipykernel_30419/874155052.py:152: UserWarning: No events found in the events file for LOAD3_DELAY4.5
  warnings.warn(f"No events found in the events file for {ev_name}")
/tmp/ipykernel_30419/874155052.py:152: UserWarning: No events found in the events file for LOAD3_DELAY4.5_rt
  warnings.warn(f"No events found in the events file for {ev_name}")
/tmp/ipykernel_30419/874155052.py:211: UserWarning: No events found for load 3 and delay 4.5
  warnings.warn(
/tmp/ipykernel_30419/874155052.py:152: UserWarning: No events found in the events file for LOAD3_DELAY4.5
  warnings.warn(f"No events found in the events file for {ev_name}")
/tmp/ipykernel_30419/874155052.py:152: UserWarning: No events found in the events file for LOAD3_DELAY4.5_rt
  warnings.warn(f"No events found in the events file for {ev_name}")


250917-19:01:40,786 nipype.workflow INFO:
	 Workflow first_level settings: ['check', 'execution', 'logging', 'monitoring']
250917-19:01:40,789 nipype.workflow INFO:
	 Running in parallel.
250917-19:01:40,790 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 54.38/54.38, Free processors: 24/24, Free GPU slot:1/1.
250917-19:01:40,917 nipype.workflow INFO:
	 [Node] Outdated cache found for "first_level.apply_mask".
250917-19:01:40,918 nipype.workflow INFO:
	 [Node] Outdated cache found for "first_level.apply_mask".
250917-19:01:40,994 nipype.workflow INFO:
	 [Node] Setting-up "first_level.apply_mask" in "/home/rongfei/WorkSpace/consortium/working/sub-10998/scap/first_level/apply_mask".
250917-19:01:40,996 nipype.workflow INFO:
	 [Node] Outdated cache found for "first_level.apply_mask".
250917-19:01:40,998 nipype.workflow INFO:
	 [Node] Executing "apply_mask" <nipype.interfaces.fsl.maths.ApplyMask>
250917-19:01:42,791 nipype.workflow INFO:
	 [MultiPro

  1%|▏         | 1/70 [02:36<2:59:41, 156.25s/it]

...

 97%|█████████▋| 68/70 [2:59:52<05:37, 168.55s/it]

/home/rongfei/WorkSpace/consortium/derivatives/sub-10339/func/sub-10339_task-scap_bold_space-MNI152NLin2009cAsym_preproc.nii.gz
/home/rongfei/WorkSpace/consortium/derivatives/sub-10339/func/sub-10339_task-scap_bold_space-MNI152NLin2009cAsym_brainmask.nii.gz
/home/rongfei/WorkSpace/consortium/derivatives/sub-10339/func/sub-10339_task-scap_bold_space-MNI152NLin2009cAsym_preproc.nii.gz
250917-22:01:33,663 nipype.workflow INFO:
	 Generated workflow graph: working/sub-10339/scap/first_level/graph.png (graph2use=colored, simple_form=True).
250917-22:01:33,679 nipype.workflow INFO:
	 Workflow first_level settings: ['check', 'execution', 'logging', 'monitoring']
250917-22:01:33,683 nipype.workflow INFO:
	 Running in parallel.
250917-22:01:33,684 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 54.38/54.38, Free processors: 24/24, Free GPU slot:1/1.
250917-22:01:33,873 nipype.workflow INFO:
	 [Node] Setting-up "first_level.apply_mask" in "/home/rongfei/Wo

 99%|█████████▊| 69/70 [3:02:35<02:46, 166.66s/it]

/home/rongfei/WorkSpace/consortium/derivatives/sub-70086/func/sub-70086_task-scap_bold_space-MNI152NLin2009cAsym_preproc.nii.gz
/home/rongfei/WorkSpace/consortium/derivatives/sub-70086/func/sub-70086_task-scap_bold_space-MNI152NLin2009cAsym_brainmask.nii.gz
/home/rongfei/WorkSpace/consortium/derivatives/sub-70086/func/sub-70086_task-scap_bold_space-MNI152NLin2009cAsym_preproc.nii.gz
250917-22:04:15,900 nipype.workflow INFO:
	 Generated workflow graph: working/sub-70086/scap/first_level/graph.png (graph2use=colored, simple_form=True).
250917-22:04:15,922 nipype.workflow INFO:
	 Workflow first_level settings: ['check', 'execution', 'logging', 'monitoring']
250917-22:04:15,925 nipype.workflow INFO:
	 Running in parallel.
250917-22:04:15,926 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 54.38/54.38, Free processors: 24/24, Free GPU slot:1/1.
250917-22:04:16,119 nipype.workflow INFO:
	 [Node] Setting-up "first_level.apply_mask" in "/home/rongfei/Wo

100%|██████████| 70/70 [3:05:23<00:00, 158.91s/it]
